In [ ]:
# Retrieves GAIA/KIC data on stellar properties

In [81]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [69]:
junk = 'KIC|Mass|E_Mass|e_Mass|Teff|E_Teff|e_Teff|logg|E_logg|e_logg|[Fe/H]|E_[Fe/H]|e_[Fe/H]|Rad|E_Rad|e_Rad|rho|E_rho|e_rho|Lum|E_Lum|e_Lum|Age|f_Age|E_Age|e_Age|Dist|E_Dist|e_Dist|Avmag|GOF|TAMS'

temp = ''
cols = []

for char in junk:
    if char == '|':
        cols.append(temp)
        temp = ''
    else:
        temp += char
cols.append(temp)

In [110]:
df = pd.read_csv("../data/asu.tsv", usecols=cols, sep='|', comment='#')
df = df.iloc[2:].reset_index(drop=True)

labels = pd.read_csv("../output/labels.csv")

In [111]:
final = pd.merge(df, labels[['KIC']], how='inner', on='KIC')
labels = pd.merge(labels, final[['KIC']], how='inner', on='KIC')

final['f_Age'] = final['f_Age'].replace(' ', 0).replace('*', 1)
final = final.astype(float)
final['KIC'] = final['KIC'].astype(int)

In [112]:
final.to_csv("../output/rf_train.csv", index=False)
labels.to_csv("../output/rf_labels.csv", index=False)